In [1]:
def the_edge_scarp(keyword):
    import webbrowser
    import requests
    from bs4 import BeautifulSoup
    import os
    import re
    import time
    from datetime import datetime, timedelta
    todaysdate=datetime.now().strftime("%Y-%m-%d")
    last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
    import warnings
    warnings.filterwarnings('ignore')
    from fuzzywuzzy import fuzz    
    import pandas as pd
    import numpy as np
    import logging

    url_base=[]
    headline=[]
    headline_url=[]
    cat=[]

    for key in keyword:
        key1=key.replace(" ","+")
        base_url="https://www.theedgemarkets.com/search-results?keywords={}&fromDate={}&toDate={}".format(key1,last_date,todaysdate)
        time.sleep(3)
        try:
            page1 = requests.get(base_url)
            soup2 = BeautifulSoup(page1.text, 'html.parser')     
            if(soup2.find('div',class_="common-grid-view view-display-id-page")!=None):
                ab = soup2.find('div',class_="common-grid-view view-display-id-page")       
                p_tag=ab.find_all("a",href=True)
                for i in p_tag:
                    head_url=i.get("href")
                    headline_url.append(head_url)
                    head_title=i.text.strip()
                    headline.append(head_title)
                    url_base.append(base_url)
                    cat.append(key)
            else:
                print("No result found")
                headline_url.append("No url found")
                headline.append("No headline found")
                url_base.append(base_url)
                cat.append(key) 
            logging.debug('Headline scrap successfully')
        except Exception as ex:
            logging.exception('Error while scraping news Headline', ex)
            pass

    df = pd.DataFrame({'keyword':cat,'main_url':url_base, 'Headline':headline,'headline_url': headline_url })
    df.drop_duplicates(inplace=True) 
    
    import nltk
    import webbrowser
    import re
    import time
    from newspaper import Article
    from newspaper import Config
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
    config = Config()
    config.browser_user_agent = user_agent
    title=[]
    summary=[]
    article=[]
    key=[]
    date_time=[]
    url_1=[]
    link='https://www.theedgemarkets.com'
    if len(df)>0:
        for url in df.headline_url:
            if url!='No url found':
                url=link+url
                print(url)
                time.sleep(5)
                url_1.append(url)
                toi_article = Article(url, language="en",config=config) # en for English
                toi_article.download()
                toi_article.parse()
                toi_article.nlp()
                title.append(toi_article.title)
                article.append(toi_article.text)
                summary.append(toi_article.summary)
                key.append(toi_article.keywords)
                date_time.append(toi_article.publish_date)

            else:
                url_1.append(url)
                title.append("")
                article.append("")
                summary.append("")
                key.append("")
                date_time.append("")

        df["article"]=article
        df["date_time"]=date_time
        df["title"]=title
        df["summary"]=summary
        df["key"]=key
        df["url"]=url_1
        df["source"]="theedgemarkets"  
        df2=df[['keyword','source','url','date_time','title','summary','article']].copy()
        df2['date_time'] = df2["date_time"].apply(lambda x : datetime.strptime(str(x).split()[0], '%Y-%m-%d') if x!='' else None)
        df2['Match'] = df2.apply(lambda x: fuzz.partial_ratio(x['keyword'], x['article']), axis=1)
        df2['Match_1'] = df2.apply(lambda x: fuzz.token_set_ratio(x['keyword'], x['article']), axis=1)
        logging.debug('Headline And Articles scrap successfully')
    else:
        df2=pd.DataFrame()
        logging.debug('Error while scraping Article')
    return df2

In [3]:
keyword=['RP TRADING','cimb bank','rhb bank']
df_the_edge=the_edge_scarp(keyword)

C:\Users\ADMIN\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


No result found


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


https://www.theedgemarkets.com/article/cimb-falls-6-post-4q-earnings-release
https://www.theedgemarkets.com/article/analysts-raise-tp-rhb-after-record-high-dividend-payout
https://www.theedgemarkets.com/article/bursa-malaysia-turns-red-midmorning
https://www.theedgemarkets.com/article/money-laundering-former-company-owner-jailed-three-years-fined-rm114-mil
https://www.theedgemarkets.com/article/affin-bank-abmb-ata-ims-bimb-bcorp-boustead-capital-cimb-ea-technique-hlb-hlfg-fgv-mahb-mah
https://www.theedgemarkets.com/article/no-material-impact-fy22-double-crediting-error-provision-says-cimb
https://www.theedgemarkets.com/article/analysts-advise-investors-accumulate-ammb-attractive-valuation
https://www.theedgemarkets.com/article/plantation-counters-give-bursa-malaysia-boost-midmorning
https://www.theedgemarkets.com/article/banks-support-og-players-together-latters-shareholders-says-maybank-ceo
https://www.theedgemarkets.com/article/malaysia-debt-turns-emerging-market-haven-fed-hikes-near

In [4]:
df_the_edge

,keyword,source,url,date_time,title,summary,article,Match,Match_1
0,RP TRADING,theedgemarkets,No url found,,,,,0,0
1,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/cimb-fa...,2022-03-01 14:14:37+08:00,CIMB falls 6% post 4Q earnings release,The counter opened at RM5.71 and slipped after...,KUALA LUMPUR (March 1): CIMB Group Holdings Bh...,44,100
2,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/analyst...,2022-03-01 14:02:07+08:00,Analysts raise TP for RHB after record high di...,RHB’s total dividend for FY21 was 40 sen versu...,KUALA LUMPUR (March 1): Analysts have raised t...,33,100
3,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/bursa-m...,2022-03-01 11:39:25+08:00,Bursa Malaysia turns red at mid-morning,KUALA LUMPUR (March 1): Bursa Malaysia’s upwar...,KUALA LUMPUR (March 1): Bursa Malaysia’s upwar...,56,100
4,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/money-l...,2022-03-01 03:36:44+08:00,Money laundering: Former company owner jailed ...,Judge Yang Zaimey Yang Ghazali pronounced sent...,SHAH ALAM (Feb 28): A former printing company ...,56,100
5,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/affin-b...,2022-03-01 01:38:38+08:00,"Affin Bank, ABMB, ATA IMS, BIMB, BCorp, Bouste...",KUALA LUMPUR (Feb 28): Based on corporate anno...,KUALA LUMPUR (Feb 28): Based on corporate anno...,56,100
6,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/no-mate...,2022-02-28 19:25:59+08:00,No material impact in FY22 from double crediti...,"At the same time, immediate actions had been t...",KUALA LUMPUR (Feb 28): CIMB Group Holdings Bhd...,33,100
7,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/analyst...,2022-02-28 13:19:23+08:00,Analysts advise investors to accumulate AMMB o...,KUALA LUMPUR (Feb 28): Analysts on Monday (Feb...,KUALA LUMPUR (Feb 28): Analysts on Monday (Feb...,33,100
8,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/plantat...,2022-02-28 12:08:48+08:00,Plantation counters give Bursa Malaysia a boos...,KUALA LUMPUR (Feb 28): Bursa Malaysia was high...,KUALA LUMPUR (Feb 28): Bursa Malaysia was high...,44,100
9,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/banks-s...,2022-02-24 17:35:20+08:00,Banks support O&G players together with the la...,He was commenting on a question citing a news ...,KUALA LUMPUR (Feb 24): Malaysia’s top lender M...,44,100


In [67]:
df_the_edge.sort_values('date_time_1',ascending=True)

,keyword,source,url,date_time,title,summary,article,Match,Match_1,date_time_1,trigger,Match_3
14,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/bursa-m...,2022-02-23 11:40:23+08:00,Bursa Malaysia stays higher at mid-morning,KUALA LUMPUR (Feb 23): Bursa Malaysia remained...,KUALA LUMPUR (Feb 23): Bursa Malaysia remained...,44,100,2022-02-23,N,11
13,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/week-ah...,2022-02-23 12:00:00+08:00,The Week Ahead: Focus on Malaysia’s January in...,Core inflation inched past 1% y-o-y for the fi...,All eyes are on Malaysia’s inflation data for ...,33,100,2022-02-23,N,33
29,rhb bank,theedgemarkets,https://www.theedgemarkets.com/article/cover-s...,2022-02-24 14:15:00+08:00,Cover Story: Softer corporate earnings seen in...,"Compared with 2020 and 2021, he says, corporat...","FACED with the ongoing Omicron wave, which saw...",38,100,2022-02-24,N,38
28,rhb bank,theedgemarkets,https://www.theedgemarkets.com/article/banks-s...,2022-02-24 17:35:20+08:00,Banks support O&G players together with the la...,He was commenting on a question citing a news ...,KUALA LUMPUR (Feb 24): Malaysia’s top lender M...,50,100,2022-02-24,N,12
12,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/malaysi...,2022-02-24 10:22:22+08:00,Malaysia on good track to achieve economic pro...,Based on reports published by the Department o...,PHNOM PENH (Feb 24): Malaysia’s economic devel...,33,100,2022-02-24,N,44
10,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/malaysi...,2022-02-24 16:10:24+08:00,Malaysia debt turns into emerging market haven...,(Feb 24): Investors are flocking to Malaysian ...,(Feb 24): Investors are flocking to Malaysian ...,22,100,2022-02-24,N,22
9,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/banks-s...,2022-02-24 17:35:20+08:00,Banks support O&G players together with the la...,He was commenting on a question citing a news ...,KUALA LUMPUR (Feb 24): Malaysia’s top lender M...,44,100,2022-02-24,N,22
11,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/cover-s...,2022-02-24 14:15:00+08:00,Cover Story: Softer corporate earnings seen in...,"Compared with 2020 and 2021, he says, corporat...","FACED with the ongoing Omicron wave, which saw...",33,100,2022-02-24,N,33
27,rhb bank,theedgemarkets,https://www.theedgemarkets.com/article/ringgit...,2022-02-26 10:07:23+08:00,Ringgit set to trade from 4.19-4.21 against US...,KUALA LUMPUR (Feb 26): The ringgit is expected...,KUALA LUMPUR (Feb 26): The ringgit is expected...,25,100,2022-02-26,N,12
7,cimb bank,theedgemarkets,https://www.theedgemarkets.com/article/analyst...,2022-02-28 13:19:23+08:00,Analysts advise investors to accumulate AMMB o...,KUALA LUMPUR (Feb 28): Analysts on Monday (Feb...,KUALA LUMPUR (Feb 28): Analysts on Monday (Feb...,33,100,2022-02-28,N,22


In [72]:
df_the_edge.iloc[29]

keyword                                                 rhb bank
source                                            theedgemarkets
url            https://www.theedgemarkets.com/article/cover-s...
date_time                              2022-02-24 14:15:00+08:00
title          Cover Story: Softer corporate earnings seen in...
summary        Compared with 2020 and 2021, he says, corporat...
article        FACED with the ongoing Omicron wave, which saw...
Match                                                         38
Match_1                                                      100
date_time_1                                  2022-02-24 00:00:00
trigger                                                        N
Match_3                                                       38
Name: 29, dtype: object

In [73]:
print(df_the_edge['url'][29])

https://www.theedgemarkets.com/article/cover-story-softer-corporate-earnings-seen-first-half


In [66]:
import warnings
warnings.filterwarnings('ignore')
from fuzzywuzzy import fuzz 
df_the_edge['Match_3'] = df_the_edge.apply(lambda x: fuzz.partial_ratio(x['keyword'].lower(), x['article'].lower()), axis=1)

In [45]:
import time
from datetime import datetime, timedelta
todaysdate=datetime.now().strftime("%Y-%m-%d")
last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")

df_the_edge['trigger']=np.where(((df_the_edge['date_time_1']>=last_date)
                                 &(df_the_edge['date_time_1']<=todaysdate))
                                & (df_the_edge['Match_1']>90),'Y','N')

In [47]:
todaysdate,last_date

('2022-03-01', '2022-02-23')

In [76]:
df_the_edge['trigger']=np.where(((df_the_edge['date_time_1']>=last_date)
                                 &(df_the_edge['date_time_1']<=todaysdate))
                                & (df_the_edge['Match_1']>90),'Y','N')

In [77]:
df_the_edge['trigger']

0     N
1     Y
2     Y
3     Y
4     Y
5     Y
6     Y
7     Y
8     Y
9     Y
10    Y
11    Y
12    Y
13    Y
14    Y
15    Y
16    Y
17    Y
18    Y
19    Y
20    Y
21    Y
22    Y
23    Y
24    Y
25    N
26    Y
27    Y
28    Y
29    Y
Name: trigger, dtype: object

In [12]:
df2.to_csv("theedgemarkets.csv",index=False)